In [283]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path

In [284]:
MAIN_DIR = Path(".").absolute()
BASEBALL_DIR = MAIN_DIR / "baseball_data"
POLIT_DIR = MAIN_DIR / "political_data"

In [285]:
teams_df = pd.read_csv(BASEBALL_DIR / "core" / "Teams.csv")
players_df = pd.read_csv(BASEBALL_DIR / "core" / "People.csv")
batting_df = pd.read_csv(BASEBALL_DIR / "core" / "Batting.csv")
pitching_df = pd.read_csv(BASEBALL_DIR / "core" / "Pitching.csv")
appearances_df = pd.read_csv(BASEBALL_DIR / "core" / "Appearances.csv")
batting_advanced_df = pd.read_csv(BASEBALL_DIR / 'advanced' / 'batting_advanced.csv')
pitching_advanced_df = pd.read_csv(BASEBALL_DIR / 'advanced' / 'pitching_advanced.csv')

In [336]:
pres_df = pd.read_csv(POLIT_DIR / "Presidents.csv").rename({"Year": "yearID"}, axis=1).drop("Republican", axis=1)
congress_df = pd.read_csv(POLIT_DIR / "Representatives.csv").rename({"Year": "yearID"}, axis=1)
senators_df = pd.read_csv(POLIT_DIR / "Senators.csv").rename({"Year": "yearID"}, axis=1)

In [286]:
batting_df = batting_df[batting_df["yearID"] >= 1899]

In [287]:
batting_df

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
7566,anderjo01,1899,1,BRO,NL,117,439,65,118,18,...,92.0,25.0,NaN,27,24.0,NaN,4.0,2.0,NaN,NaN
7567,atherch01,1899,1,WAS,NL,65,242,28,60,5,...,23.0,2.0,NaN,21,26.0,NaN,2.0,4.0,NaN,NaN
7568,baileha01,1899,1,BSN,NL,12,34,3,8,2,...,2.0,0.0,NaN,2,3.0,NaN,1.0,0.0,NaN,NaN
7569,bakerki01,1899,1,WAS,NL,12,19,1,3,0,...,1.0,0.0,NaN,1,6.0,NaN,0.0,0.0,NaN,NaN
7570,barreji01,1899,1,CIN,NL,26,92,30,34,2,...,10.0,4.0,NaN,18,7.0,NaN,1.0,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110490,zimmejo02,2021,1,MIL,NL,2,1,0,0,0,...,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
110491,zimmeky01,2021,1,KCA,AL,52,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
110492,zimmery01,2021,1,WAS,NL,110,255,27,62,16,...,46.0,0.0,0.0,16,77.0,0.0,0.0,0.0,2.0,9.0
110493,zuberty01,2021,1,KCA,AL,31,1,0,0,0,...,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0


In [288]:
teams_df["minAB"] = 3.1 * teams_df['G']

In [289]:
batting_heavy_min = pd.merge(batting_df, teams_df[['minAB', 'yearID', 'teamID']], on=['yearID', 'teamID'])

In [290]:
batting_heavy_min["PA"] = batting_heavy_min[["AB", "HBP", "BB", "SF", "SH"]].sum(axis=1, skipna=True)

In [291]:
batting_heavy_min = batting_heavy_min[batting_heavy_min["minAB"] < batting_heavy_min["PA"]]

In [292]:
batting_heavy_data = pd.merge(batting_heavy_min, players_df, on=["playerID"])
batting_heavy_data["fullName"] = batting_heavy_data["nameFirst"] + " " + batting_heavy_data["nameLast"]

In [293]:
batting_heavy_advanced = pd.merge(batting_heavy_data,batting_advanced_df.drop("G", axis=1), on=['yearID','bbrefID'])

In [294]:
batting_heavy_advanced_final = batting_heavy_advanced[['playerID', 'bbrefID', 'yearID', 'fullName', 'teamID', 'birthState', 'birthCountry', 'G','AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'HBP', 'SH', 'SF', 'WAR', 'oWAR', 'dWAR']]

In [295]:
batting_heavy_advanced_final

,playerID,bbrefID,yearID,fullName,teamID,birthState,birthCountry,G,AB,R,...,SB,CS,BB,SO,HBP,SH,SF,WAR,oWAR,dWAR
0,anderjo01,anderjo01,1899,John Anderson,BRO,NaN,Norway,117,439,65,...,25.0,NaN,27,24.0,4.0,2.0,NaN,0.6,0.4,-0.2
1,anderjo01,anderjo01,1901,John Anderson,MLA,NaN,Norway,138,576,90,...,35.0,NaN,24,21.0,3.0,4.0,NaN,4.4,4.3,-0.2
2,anderjo01,anderjo01,1902,John Anderson,SLA,NaN,Norway,126,524,60,...,15.0,NaN,21,9.0,3.0,3.0,NaN,0.9,1.3,-0.6
3,anderjo01,anderjo01,1903,John Anderson,SLA,NaN,Norway,138,550,65,...,16.0,NaN,23,39.0,0.0,4.0,NaN,3.3,2.5,0.4
4,anderjo01,anderjo01,1904,John Anderson,NYA,NaN,Norway,143,558,62,...,20.0,NaN,23,37.0,6.0,11.0,NaN,3.2,2.8,-0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12251,lowena01,lowena01,2021,Nathaniel Lowe,TEX,VA,USA,157,557,75,...,8.0,0.0,80,162.0,2.0,0.0,3.0,2.4,2.6,-1.1
12252,francty01,francty01,2021,Ty France,SEA,CA,USA,152,571,85,...,0.0,0.0,46,106.0,27.0,0.0,6.0,4.3,4.1,-0.5
12253,hernaen02,hernaen02,2021,Enrique Hernandez,BOS,NaN,P.R.,134,508,84,...,1.0,0.0,61,110.0,9.0,0.0,7.0,4.9,3.2,2.1
12254,renfrhu01,renfrhu01,2021,Hunter Renfroe,BOS,MS,USA,144,521,89,...,1.0,2.0,44,130.0,1.0,0.0,6.0,2.3,2.3,-0.5


In [296]:
corrMatrix = batting_heavy_advanced_final.corr()
px.imshow(corrMatrix, text_auto=True, aspect='auto', zmax=1, zmin=-1, color_continuous_scale=px.colors.diverging.Fall)

In [338]:
batting_political = pd.merge(batting_heavy_advanced_final, pres_df, on="yearID").rename({"Democrat": "demPres"},
                                                                                         axis=1)
batting_political = pd.merge(batting_political,
                              senators_df[["yearID", "% Democrats"]],
                              on="yearID").rename({"% Democrats": "demSenate"}, axis=1)

batting_political = pd.merge(batting_political,
                              congress_df[["yearID", "% Democrats"]],
                              on="yearID").rename({"% Democrats": "demCongress"}, axis=1)
batting_political

,playerID,bbrefID,yearID,fullName,teamID,birthState,birthCountry,G,AB,R,...,SO,HBP,SH,SF,WAR,oWAR,dWAR,demPres,demSenate,demCongress
0,anderjo01,anderjo01,1899,John Anderson,BRO,NaN,Norway,117,439,65,...,24.0,4.0,2.0,NaN,0.6,0.4,-0.2,0,0.29,0.457
1,caseydo01,caseydo01,1899,Doc Casey,BRO,MA,USA,134,525,75,...,16.0,9.0,6.0,NaN,-0.5,-0.3,-0.2,0,0.29,0.457
2,caseydo01,caseydo01,1899,Doc Casey,BRO,MA,USA,134,525,75,...,16.0,9.0,6.0,NaN,0.5,0.8,0.2,0,0.29,0.457
3,dahlebi01,dahlebi01,1899,Bill Dahlen,BRO,NY,USA,121,428,87,...,23.0,15.0,4.0,NaN,4.6,3.7,1.7,0,0.29,0.457
4,dalyto01,dalyto01,1899,Tom Daly,BRO,PA,USA,141,498,95,...,34.0,12.0,8.0,NaN,4.6,4.3,0.5,0,0.29,0.457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12251,hanigmi01,hanigmi01,2018,Mitch Haniger,SEA,CA,USA,157,596,90,...,148.0,10.0,0.0,7.0,6.5,5.5,0.5,0,0.46,0.446
12252,chapmma01,chapmma01,2018,Matt Chapman,OAK,CA,USA,145,547,100,...,146.0,9.0,0.0,2.0,7.6,5.1,2.9,0,0.46,0.446
12253,olsonma02,olsonma02,2018,Matt Olson,OAK,GA,USA,162,580,85,...,163.0,8.0,0.0,2.0,3.7,2.7,0.2,0,0.46,0.446
12254,confomi01,confomi01,2018,Michael Conforto,NYN,WA,USA,153,543,78,...,159.0,7.0,0.0,4.0,2.7,3.4,-1.0,0,0.46,0.446


In [339]:
idx = pd.MultiIndex.from_product((batting_political['playerID'].unique(), batting_political['demPres'].unique()),
                                 names=["playerID", "demPres"])
bat_pres_comp = batting_political.groupby(["playerID", "demPres"]).agg({"WAR": "mean"}).reindex(idx)
bat_pres_comp = bat_pres_comp.fillna(0)
batting_political_score = bat_pres_comp.groupby("playerID").agg(np.subtract.reduce)
batting_political_score.columns = ['Political Score']
batting_political_score

,Political Score
playerID,
aaronha01,-2.195833
abbated01,1.680000
abreubo01,0.295238
abreujo02,-0.750000
abstebi01,0.800000
...,...
zimmehe01,2.233333
zimmery01,-1.800000
ziskri01,1.733333


Pitch

In [297]:
pitching_df = pitching_df[pitching_df["yearID"] >= 1899]
teams_df["minOUT"] = teams_df["G"] * 3
pitching_min = pd.merge(pitching_df, teams_df[['minOUT', 'yearID', 'teamID']], on=['yearID', 'teamID'])
pitching_min = pitching_min[pitching_min["minOUT"] < pitching_min["IPouts"]]

In [298]:
pitching_full = pd.merge(pitching_min, players_df, on=["playerID"])
pitching_full["fullName"] = pitching_full["nameFirst"] + " " + pitching_full["nameLast"]

In [299]:
pitching_full

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,fullName
0,lewiste01,1899,1,BSN,NL,17,11,29,25,23,...,Edward Morgan,158.0,70.0,R,R,1896-07-06,1901-09-26,lewit101,lewiste01,Ted Lewis
1,lewiste01,1900,1,BSN,NL,13,12,30,22,19,...,Edward Morgan,158.0,70.0,R,R,1896-07-06,1901-09-26,lewit101,lewiste01,Ted Lewis
2,lewiste01,1901,1,BOS,AL,16,17,39,34,31,...,Edward Morgan,158.0,70.0,R,R,1896-07-06,1901-09-26,lewit101,lewiste01,Ted Lewis
3,nichoki01,1899,1,BSN,NL,21,19,42,37,37,...,Charles Augustus,175.0,70.0,B,R,1890-04-23,1906-05-18,nichk101,nichoki01,Kid Nichols
4,nichoki01,1900,1,BSN,NL,13,16,29,27,25,...,Charles Augustus,175.0,70.0,B,R,1890-04-23,1906-05-18,nichk101,nichoki01,Kid Nichols
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8547,uriasju01,2021,1,LAN,NL,20,3,32,32,0,...,Julio Cesar,225.0,72.0,L,L,2016-05-27,2021-10-02,uriaj001,uriasju01,Julio Urias
8548,lylesjo01,2021,1,TEX,AL,10,13,32,30,0,...,Jordan Horton,230.0,77.0,R,R,2011-05-31,2021-10-02,lylej001,lylesjo01,Jordan Lyles
8549,burneco01,2021,1,MIL,NL,11,5,28,28,0,...,Corbin Brian,225.0,75.0,R,R,2018-07-10,2021-10-02,burnc002,burneco01,Corbin Burnes
8550,flexech01,2021,1,SEA,AL,14,6,31,31,0,...,Christopher John,250.0,75.0,R,R,2017-07-27,2021-10-02,flexc001,flexech01,Chris Flexen


In [300]:
pitching_full_advanced = pd.merge(pitching_full,pitching_advanced_df.drop(['G','GS'], axis=1), on=['yearID','bbrefID'])
pitching_full_advanced

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,WAA,gmLI,WAAadj,WAR,RAR,waaWL%,162WL%,Salary,Awards,RA9extras
0,lewiste01,1899,1,BSN,NL,17,11,29,25,23,...,1.6,1.00,-0.2,3.6,37.0,0.555,0.510,NaN,NaN,NaN
1,lewiste01,1900,1,BSN,NL,13,12,30,22,19,...,0.8,1.00,-0.1,2.7,28.0,0.527,0.505,NaN,NaN,NaN
2,lewiste01,1901,1,BOS,AL,16,17,39,34,31,...,0.1,1.00,-0.2,3.2,40.0,0.504,0.501,NaN,NaN,NaN
3,nichoki01,1899,1,BSN,NL,21,19,42,37,37,...,4.3,1.00,-0.4,7.3,74.0,0.603,0.527,"$3,000",NaN,NaN
4,nichoki01,1900,1,BSN,NL,13,16,29,27,25,...,2.8,1.00,-0.2,4.8,50.0,0.596,0.517,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8733,uriasju01,2021,1,LAN,NL,20,3,32,32,0,...,2.8,NaN,-0.2,4.4,43.0,0.588,0.517,"$3,600,000",CYA-7,0.0
8734,lylesjo01,2021,1,TEX,AL,10,13,32,30,0,...,-1.6,0.55,-0.1,-0.1,2.0,0.449,0.490,"$8,000,000",NaN,0.0
8735,burneco01,2021,1,MIL,NL,11,5,28,28,0,...,4.1,NaN,-0.1,5.6,52.0,0.648,0.526,NaN,"AS,CYA-1,MVP-15",0.0
8736,flexech01,2021,1,SEA,AL,14,6,31,31,0,...,1.9,NaN,-0.1,3.5,37.0,0.562,0.512,"$1,700,000",NaN,0.0


In [301]:
pitching_full_advanced_final = pitching_full_advanced[['playerID', 'bbrefID', 'yearID', 'fullName', 'teamID', 'birthState', 'birthCountry', 'W', 'GS', 'G', 'SV', 'ER', 'SO', 'BB', 'ERA', 'BAOpp', 'WP', 'HR', 'gmLI', 'WAR']]

In [326]:
pitching_full_advanced_final


,playerID,bbrefID,yearID,fullName,teamID,birthState,birthCountry,W,GS,G,SV,ER,SO,BB,ERA,BAOpp,WP,HR,gmLI,WAR
0,lewiste01,lewiste01,1899,Ted Lewis,BSN,Powys,United Kingdom,17,25,29,0,91,60,73,3.49,NaN,3,10,1.00,3.6
1,lewiste01,lewiste01,1900,Ted Lewis,BSN,Powys,United Kingdom,13,22,30,0,96,66,86,4.13,NaN,8,11,1.00,2.7
2,lewiste01,lewiste01,1901,Ted Lewis,BOS,Powys,United Kingdom,16,34,39,1,124,103,91,3.53,NaN,2,14,1.00,3.2
3,nichoki01,nichoki01,1899,Kid Nichols,BSN,WI,USA,21,37,42,1,114,108,82,2.99,NaN,8,11,1.00,7.3
4,nichoki01,nichoki01,1900,Kid Nichols,BSN,WI,USA,13,27,29,0,79,53,72,3.07,NaN,6,11,1.00,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8733,uriasju01,uriasju01,2021,Julio Urias,LAN,Sinaloa,Mexico,20,32,32,0,61,195,38,2.96,0.219,2,19,NaN,4.4
8734,lylesjo01,lylesjo01,2021,Jordan Lyles,TEX,SC,USA,10,30,32,0,103,146,56,5.15,0.278,9,38,0.55,-0.1
8735,burneco01,burneco01,2021,Corbin Burnes,MIL,CA,USA,11,28,28,0,45,234,34,2.43,0.201,5,7,NaN,5.6
8736,flexech01,flexech01,2021,Chris Flexen,SEA,CA,USA,14,31,31,0,72,125,40,3.61,0.268,2,19,NaN,3.5


In [303]:
corrMatrix = pitching_full_advanced_final.corr()
px.imshow(corrMatrix, text_auto=True, aspect='auto', zmax=1, zmin=-1, color_continuous_scale=px.colors.diverging.Fall)

In [337]:
pitching_political = pd.merge(pitching_full_advanced_final, pres_df, on="yearID").rename({"Democrat": "demPres"},
                                                                                         axis=1)
pitching_political = pd.merge(pitching_political,
                              senators_df[["yearID", "% Democrats"]],
                              on="yearID").rename({"% Democrats": "demSenate"}, axis=1)

pitching_political = pd.merge(pitching_political,
                              congress_df[["yearID", "% Democrats"]],
                              on="yearID").rename({"% Democrats": "demCongress"}, axis=1)
pitching_political

,playerID,bbrefID,yearID,fullName,teamID,birthState,birthCountry,W,GS,G,...,BB,ERA,BAOpp,WP,HR,gmLI,WAR,demPres,demSenate,demCongress
0,lewiste01,lewiste01,1899,Ted Lewis,BSN,Powys,United Kingdom,17,25,29,...,73,3.49,NaN,3,10,1.00,3.6,0,0.29,0.457
1,nichoki01,nichoki01,1899,Kid Nichols,BSN,WI,USA,21,37,42,...,82,2.99,NaN,8,11,1.00,7.3,0,0.29,0.457
2,willivi01,willivi01,1899,Vic Willis,BSN,MD,USA,27,38,41,...,117,2.50,NaN,12,6,1.00,10.4,0,0.29,0.457
3,dinnebi01,dinnebi01,1899,Bill Dinneen,WAS,NY,USA,14,35,37,...,106,3.93,NaN,8,6,1.00,3.3,0,0.29,0.457
4,mcfarda01,mcfarda01,1899,Dan McFarlan,WAS,TX,USA,8,28,32,...,64,4.76,NaN,5,5,1.00,0.3,0,0.29,0.457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8733,rodried05,rodried05,2019,Eduardo Rodriguez,BOS,Carabobo,Venezuela,19,34,34,...,75,3.81,0.253,3,24,NaN,5.5,0,0.45,0.541
8734,friedma01,friedma01,2019,Max Fried,ATL,CA,USA,17,30,33,...,47,4.02,0.270,11,21,0.94,3.0,0,0.45,0.541
8735,sorokmi01,sorokmi01,2019,Mike Soroka,ATL,AB,CAN,13,29,29,...,41,2.68,0.236,3,14,NaN,6.1,0,0.45,0.541
8736,flaheja01,flaheja01,2019,Jack Flaherty,SLN,CA,USA,11,33,33,...,55,2.75,0.192,6,25,NaN,5.8,0,0.45,0.541


In [366]:
idx = pd.MultiIndex.from_product((pitching_political['playerID'].unique(), pitching_political['demPres'].unique()),
                                 names=["playerID", "demPres"])
pitch_pres_comp = pitching_political.groupby(["playerID", "demPres"]).agg({"WAR": "mean"}).reindex(idx)
pitch_pres_comp = pitch_pres_comp.fillna(0)
pitching_political_score = pitch_pres_comp.groupby("playerID").agg(np.subtract.reduce)
pitching_political_score.columns = ['Political Score']
pitching_political_score

,Political Score
playerID,
aasedo01,-1.833333
abbotgl01,-0.300000
abbotji01,2.075000
abbotpa01,-1.300000
adamsba01,-2.193333
...,...
zambrvi01,2.150000
zimmejo02,-3.780000
zitoba01,2.466667


In [304]:
from sklearn.preprocessing import StandardScaler

In [340]:
batting_heavy_advanced_poli = pd.merge(batting_heavy_advanced_final,batting_political_score,on = 'playerID')
batting_heavy_advanced_poli

,playerID,bbrefID,yearID,fullName,teamID,birthState,birthCountry,G,AB,R,...,CS,BB,SO,HBP,SH,SF,WAR,oWAR,dWAR,Political Score
0,anderjo01,anderjo01,1899,John Anderson,BRO,NaN,Norway,117,439,65,...,NaN,27,24.0,4.0,2.0,NaN,0.6,0.4,-0.2,2.383333
1,anderjo01,anderjo01,1901,John Anderson,MLA,NaN,Norway,138,576,90,...,NaN,24,21.0,3.0,4.0,NaN,4.4,4.3,-0.2,2.383333
2,anderjo01,anderjo01,1902,John Anderson,SLA,NaN,Norway,126,524,60,...,NaN,21,9.0,3.0,3.0,NaN,0.9,1.3,-0.6,2.383333
3,anderjo01,anderjo01,1903,John Anderson,SLA,NaN,Norway,138,550,65,...,NaN,23,39.0,0.0,4.0,NaN,3.3,2.5,0.4,2.383333
4,anderjo01,anderjo01,1904,John Anderson,NYA,NaN,Norway,143,558,62,...,NaN,23,37.0,6.0,11.0,NaN,3.2,2.8,-0.3,2.383333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12251,lowena01,lowena01,2021,Nathaniel Lowe,TEX,VA,USA,157,557,75,...,0.0,80,162.0,2.0,0.0,3.0,2.4,2.6,-1.1,-2.400000
12252,francty01,francty01,2021,Ty France,SEA,CA,USA,152,571,85,...,0.0,46,106.0,27.0,0.0,6.0,4.3,4.1,-0.5,-4.300000
12253,hernaen02,hernaen02,2021,Enrique Hernandez,BOS,NaN,P.R.,134,508,84,...,0.0,61,110.0,9.0,0.0,7.0,4.9,3.2,2.1,-4.900000
12254,renfrhu01,renfrhu01,2021,Hunter Renfroe,BOS,MS,USA,144,521,89,...,2.0,44,130.0,1.0,0.0,6.0,2.3,2.3,-0.5,-2.300000


In [344]:
batting_heavy_advanced_main = batting_heavy_advanced_poli.iloc[:,7:26]
batting_heavy_advanced_main

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,WAR,oWAR,dWAR,Political Score
0,117,439,65,118,18,7,4,92.0,25.0,NaN,27,24.0,4.0,2.0,NaN,0.6,0.4,-0.2,2.383333
1,138,576,90,190,46,7,8,99.0,35.0,NaN,24,21.0,3.0,4.0,NaN,4.4,4.3,-0.2,2.383333
2,126,524,60,149,29,6,4,85.0,15.0,NaN,21,9.0,3.0,3.0,NaN,0.9,1.3,-0.6,2.383333
3,138,550,65,156,34,8,2,78.0,16.0,NaN,23,39.0,0.0,4.0,NaN,3.3,2.5,0.4,2.383333
4,143,558,62,155,27,12,3,82.0,20.0,NaN,23,37.0,6.0,11.0,NaN,3.2,2.8,-0.3,2.383333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12251,157,557,75,147,24,3,18,72.0,8.0,0.0,80,162.0,2.0,0.0,3.0,2.4,2.6,-1.1,-2.400000
12252,152,571,85,166,32,1,18,73.0,0.0,0.0,46,106.0,27.0,0.0,6.0,4.3,4.1,-0.5,-4.300000
12253,134,508,84,127,35,3,20,60.0,1.0,0.0,61,110.0,9.0,0.0,7.0,4.9,3.2,2.1,-4.900000
12254,144,521,89,135,33,0,31,96.0,1.0,2.0,44,130.0,1.0,0.0,6.0,2.3,2.3,-0.5,-2.300000


In [345]:
batting_heavy_advanced_main = batting_heavy_advanced_main.fillna(value = 0,axis=0)
batting_heavy_advanced_main

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,WAR,oWAR,dWAR,Political Score
0,117,439,65,118,18,7,4,92.0,25.0,0.0,27,24.0,4.0,2.0,0.0,0.6,0.4,-0.2,2.383333
1,138,576,90,190,46,7,8,99.0,35.0,0.0,24,21.0,3.0,4.0,0.0,4.4,4.3,-0.2,2.383333
2,126,524,60,149,29,6,4,85.0,15.0,0.0,21,9.0,3.0,3.0,0.0,0.9,1.3,-0.6,2.383333
3,138,550,65,156,34,8,2,78.0,16.0,0.0,23,39.0,0.0,4.0,0.0,3.3,2.5,0.4,2.383333
4,143,558,62,155,27,12,3,82.0,20.0,0.0,23,37.0,6.0,11.0,0.0,3.2,2.8,-0.3,2.383333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12251,157,557,75,147,24,3,18,72.0,8.0,0.0,80,162.0,2.0,0.0,3.0,2.4,2.6,-1.1,-2.400000
12252,152,571,85,166,32,1,18,73.0,0.0,0.0,46,106.0,27.0,0.0,6.0,4.3,4.1,-0.5,-4.300000
12253,134,508,84,127,35,3,20,60.0,1.0,0.0,61,110.0,9.0,0.0,7.0,4.9,3.2,2.1,-4.900000
12254,144,521,89,135,33,0,31,96.0,1.0,2.0,44,130.0,1.0,0.0,6.0,2.3,2.3,-0.5,-2.300000


In [347]:
scaler = StandardScaler()
batting_heavy_advanced_main_scale = pd.DataFrame(scaler.fit_transform(batting_heavy_advanced_main),columns=['G','AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'HBP', 'SH', 'SF', 'WAR', 'oWAR', 'dWAR','Political Score'])
batting_heavy_advanced_main_scale.describe()

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,WAR,oWAR,dWAR,Political Score
count,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04,1.225600e+04
mean,-9.151187e-16,5.113516e-16,1.788792e-16,5.677822e-16,6.833633e-16,1.170195e-14,-1.890255e-15,-1.615845e-14,1.857468e-16,-1.534873e-14,2.369664e-16,-6.071152e-15,-3.209356e-15,1.452220e-14,-2.901387e-14,-1.071826e-16,-1.565237e-16,6.621843e-17,-7.719927e-16
std,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00
min,-6.195505e+00,-5.361025e+00,-2.815868e+00,-4.097632e+00,-2.713199e+00,-1.274061e+00,-1.237173e+00,-2.389256e+00,-9.563396e-01,-9.079894e-01,-2.119339e+00,-1.906789e+00,-1.095839e+00,-8.464502e-01,-9.836510e-01,-3.167905e+00,-3.076999e+00,-4.605330e+00,-3.356625e+00
25%,-4.682473e-01,-5.725492e-01,-7.011651e-01,-6.360881e-01,-7.422680e-01,-7.774752e-01,-8.840822e-01,-7.437895e-01,-7.259217e-01,-9.079894e-01,-7.347114e-01,-7.759837e-01,-5.518917e-01,-7.181469e-01,-9.836510e-01,-7.075366e-01,-6.800278e-01,-6.828193e-01,-6.862188e-01
50%,2.317508e-01,8.107063e-02,-7.636654e-02,-2.942582e-02,-4.664538e-02,-2.808894e-01,-1.778997e-01,-7.776737e-02,-3.418918e-01,-2.800632e-01,-1.289368e-01,-1.416297e-01,-2.799180e-01,-3.332369e-01,-6.194175e-03,-6.966330e-02,-1.190346e-01,-7.265095e-02,-8.933964e-04
75%,6.772042e-01,6.920631e-01,6.445549e-01,6.486085e-01,6.489772e-01,4.639892e-01,6.165557e-01,6.666103e-01,4.261678e-01,5.571718e-01,5.633770e-01,6.306274e-01,2.640292e-01,4.365831e-01,6.454437e-01,6.137723e-01,5.949567e-01,6.246843e-01,7.080639e-01
max,1.440839e+00,2.624504e+00,4.777838e+00,4.003094e+00,4.706776e+00,7.664483e+00,5.206742e+00,4.701921e+00,9.028436e+00,7.882978e+00,7.702863e+00,4.243687e+00,1.250284e+01,7.749873e+00,5.206909e+00,5.078885e+00,4.827905e+00,4.634362e+00,3.276065e+00


In [348]:
from sklearn.cluster import KMeans

In [349]:
import plotly.graph_objects as go
inertia = []
for i in range(1,11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(batting_heavy_advanced_main_scale)
    inertia.append(kmeans.inertia_)

In [350]:
fig = go.Figure(data=go.Scatter(x=np.arange(1,11),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,11],title="Cluster Number"),
                  yaxis={'title':'Inertia'},
                 annotations=[
        dict(
            x=3,
            y=inertia[2],
            xref="x",
            yref="y",
            text="Elbow!",
            showarrow=True,
            arrowhead=7,
            ax=20,
            ay=-40
        )
    ])

In [361]:
kmeans = KMeans(
        n_clusters= 4, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )

In [362]:
kmeans.fit(batting_heavy_advanced_main_scale)
batting_heavy_advanced_main_scale['label']=kmeans.labels_
polar=batting_heavy_advanced_main_scale.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
fig4 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1200)
fig4.show()

/Users/mhd/opt/anaconda3/lib/python3.8/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/mhd/opt/anaconda3/lib/python3.8/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/mhd/opt/anaconda3/lib/python3.8/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/mhd/opt/anaconda3/lib/python3.8/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [363]:
pie=batting_heavy_advanced_main_scale.groupby('label').size().reset_index()
pie.columns=['label','value']
px.pie(pie,values='value',names='label')

In [364]:
from sklearn.decomposition import PCA
# import seaborn as sn
pca_num_components = 2

reduced_data = PCA(n_components=pca_num_components).fit_transform(batting_heavy_advanced_main_scale.drop('label',axis = 1))
results = pd.DataFrame(reduced_data,columns=['pca1','pca2'])

In [365]:
px.scatter(results,x="pca1", y="pca2", color=batting_heavy_advanced_main_scale['label'],hover_name=batting_heavy_advanced_final['fullName'])

Pitch

In [372]:
pitching_full_advanced_poli = pd.merge(pitching_full_advanced_final,pitching_political_score, on = 'playerID')
pitching_full_advanced_poli

,playerID,bbrefID,yearID,fullName,teamID,birthState,birthCountry,W,GS,G,...,ER,SO,BB,ERA,BAOpp,WP,HR,gmLI,WAR,Political Score
0,lewiste01,lewiste01,1899,Ted Lewis,BSN,Powys,United Kingdom,17,25,29,...,91,60,73,3.49,NaN,3,10,1.00,3.6,3.166667
1,lewiste01,lewiste01,1900,Ted Lewis,BSN,Powys,United Kingdom,13,22,30,...,96,66,86,4.13,NaN,8,11,1.00,2.7,3.166667
2,lewiste01,lewiste01,1901,Ted Lewis,BOS,Powys,United Kingdom,16,34,39,...,124,103,91,3.53,NaN,2,14,1.00,3.2,3.166667
3,nichoki01,nichoki01,1899,Kid Nichols,BSN,WI,USA,21,37,42,...,114,108,82,2.99,NaN,8,11,1.00,7.3,6.575000
4,nichoki01,nichoki01,1900,Kid Nichols,BSN,WI,USA,13,27,29,...,79,53,72,3.07,NaN,6,11,1.00,4.8,6.575000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8733,uriasju01,uriasju01,2021,Julio Urias,LAN,Sinaloa,Mexico,20,32,32,...,61,195,38,2.96,0.219,2,19,NaN,4.4,-4.400000
8734,lylesjo01,lylesjo01,2021,Jordan Lyles,TEX,SC,USA,10,30,32,...,103,146,56,5.15,0.278,9,38,0.55,-0.1,0.100000
8735,burneco01,burneco01,2021,Corbin Burnes,MIL,CA,USA,11,28,28,...,45,234,34,2.43,0.201,5,7,NaN,5.6,-5.600000
8736,flexech01,flexech01,2021,Chris Flexen,SEA,CA,USA,14,31,31,...,72,125,40,3.61,0.268,2,19,NaN,3.5,-3.500000


In [373]:
pitching_full_advanced_main = pitching_full_advanced_poli.iloc[:, 7:21]
pitching_full_advanced_main

,W,GS,G,SV,ER,SO,BB,ERA,BAOpp,WP,HR,gmLI,WAR,Political Score
0,17,25,29,0,91,60,73,3.49,NaN,3,10,1.00,3.6,3.166667
1,13,22,30,0,96,66,86,4.13,NaN,8,11,1.00,2.7,3.166667
2,16,34,39,1,124,103,91,3.53,NaN,2,14,1.00,3.2,3.166667
3,21,37,42,1,114,108,82,2.99,NaN,8,11,1.00,7.3,6.575000
4,13,27,29,0,79,53,72,3.07,NaN,6,11,1.00,4.8,6.575000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8733,20,32,32,0,61,195,38,2.96,0.219,2,19,NaN,4.4,-4.400000
8734,10,30,32,0,103,146,56,5.15,0.278,9,38,0.55,-0.1,0.100000
8735,11,28,28,0,45,234,34,2.43,0.201,5,7,NaN,5.6,-5.600000
8736,14,31,31,0,72,125,40,3.61,0.268,2,19,NaN,3.5,-3.500000


In [374]:
pitching_full_advanced_main = pitching_full_advanced_main.fillna(value=0, axis=0)
batting_heavy_advanced_main

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,HBP,SH,SF,WAR,oWAR,dWAR,Political Score
0,117,439,65,118,18,7,4,92.0,25.0,0.0,27,24.0,4.0,2.0,0.0,0.6,0.4,-0.2,2.383333
1,138,576,90,190,46,7,8,99.0,35.0,0.0,24,21.0,3.0,4.0,0.0,4.4,4.3,-0.2,2.383333
2,126,524,60,149,29,6,4,85.0,15.0,0.0,21,9.0,3.0,3.0,0.0,0.9,1.3,-0.6,2.383333
3,138,550,65,156,34,8,2,78.0,16.0,0.0,23,39.0,0.0,4.0,0.0,3.3,2.5,0.4,2.383333
4,143,558,62,155,27,12,3,82.0,20.0,0.0,23,37.0,6.0,11.0,0.0,3.2,2.8,-0.3,2.383333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12251,157,557,75,147,24,3,18,72.0,8.0,0.0,80,162.0,2.0,0.0,3.0,2.4,2.6,-1.1,-2.400000
12252,152,571,85,166,32,1,18,73.0,0.0,0.0,46,106.0,27.0,0.0,6.0,4.3,4.1,-0.5,-4.300000
12253,134,508,84,127,35,3,20,60.0,1.0,0.0,61,110.0,9.0,0.0,7.0,4.9,3.2,2.1,-4.900000
12254,144,521,89,135,33,0,31,96.0,1.0,2.0,44,130.0,1.0,0.0,6.0,2.3,2.3,-0.5,-2.300000


In [375]:
scaler = StandardScaler()
pitching_full_advanced_main_scale = pd.DataFrame(scaler.fit_transform(pitching_full_advanced_main),
                                                 columns=['W', 'GS', 'G', 'SV', 'ER', 'SO', 'BB', 'ERA', 'BAOpp', 'WP', 'HR', 'gmLI', 'WAR','Political Score'])
pitching_full_advanced_main_scale.describe()

,W,GS,G,SV,ER,SO,BB,ERA,BAOpp,WP,HR,gmLI,WAR,Political Score
count,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03,8.738000e+03
mean,2.161334e-15,1.222427e-15,2.837021e-15,1.764008e-14,-2.682425e-16,-8.679510e-16,4.058045e-16,6.087613e-16,1.622593e-15,1.167195e-15,5.023473e-15,2.029355e-14,-9.601927e-17,-7.279343e-16
std,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00,1.000057e+00
min,-2.648453e+00,-5.040474e+00,-4.024341e+00,-4.775665e-01,-3.510985e+00,-1.928847e+00,-2.731654e+00,-3.063983e+00,-3.481728e-01,-1.377024e+00,-1.786607e+00,-9.389414e-01,-3.048631e+00,-3.537436e+00
25%,-6.832911e-01,-5.479507e-01,-5.139232e-01,-4.775665e-01,-6.705166e-01,-7.560734e-01,-6.713006e-01,-6.794182e-01,-9.522830e-02,-8.199279e-01,-8.402480e-01,-9.389414e-01,-6.965267e-01,-7.002997e-01
50%,-2.823723e-02,1.432068e-01,-1.243486e-02,-4.775665e-01,-1.112207e-02,-1.199930e-01,-8.902696e-02,-3.335380e-02,-7.323312e-02,-2.628321e-01,-1.218418e-02,4.699009e-02,-1.085007e-01,2.755194e-02
75%,6.268166e-01,6.615748e-01,4.890534e-01,1.876616e-01,6.482725e-01,5.955974e-01,5.828273e-01,6.479505e-01,-5.123795e-02,5.728115e-01,6.975848e-01,4.783351e-01,5.699909e-01,6.811330e-01
max,6.085599e+00,3.771783e+00,1.202328e+01,2.014451e+01,5.264034e+00,5.286690e+00,6.271193e+00,4.864989e+00,1.063842e+01,6.979412e+00,4.128135e+00,1.007063e+01,5.545596e+00,4.216413e+00


In [376]:
import plotly.graph_objects as go

inertia = []
for i in range(1, 11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(pitching_full_advanced_main_scale)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1, 11), y=inertia))
fig.update_layout(title="Inertia vs Cluster Number", xaxis=dict(range=[0, 11], title="Cluster Number"),
                  yaxis={'title': 'Inertia'},
                  annotations=[
                      dict(
                          x=3,
                          y=inertia[2],
                          xref="x",
                          yref="y",
                          text="Elbow!",
                          showarrow=True,
                          arrowhead=7,
                          ax=20,
                          ay=-40
                      )
                  ])

In [379]:
kmeans = KMeans(
        n_clusters= 3, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )

kmeans.fit(pitching_full_advanced_main_scale)
pitching_full_advanced_main_scale['label']=kmeans.labels_

from sklearn.decomposition import PCA

# import seaborn as sn
pca_num_components = 2

reduced_data = PCA(n_components=pca_num_components).fit_transform(
   pitching_full_advanced_main_scale.drop('label', axis=1))
results = pd.DataFrame(reduced_data, columns=['pca1', 'pca2'])
px.scatter(results, x="pca1", y="pca2", color=pitching_full_advanced_main_scale['label'],
           hover_name=pitching_full_advanced_poli['fullName'])